# VacationPy
___

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load cities.csv to a DataFrame

cities_data = pd.read_csv("Cities.csv")

# Make sure to handle NaN values

cities_data = cities_data.dropna()

cities_data = cities_data.set_index("City_ID")

cities_data

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
City_ID,,,,,,,,
0,Kapaa,US,22.08,-159.32,80.60,61,40,10.29
1,Newala,TZ,-10.94,39.28,64.76,100,41,4.14
2,Tuktoyaktuk,CA,69.45,-133.04,30.20,80,90,8.05
3,Busselton,AU,-33.65,115.33,48.00,98,68,25.50
4,Yerofey pavlovich,RU,53.97,121.97,43.09,57,1,0.69
...,...,...,...,...,...,...,...,...
483,Grand-santi,GF,4.25,-54.38,76.08,96,99,3.24
484,Yakeshi,CN,49.28,120.73,46.42,54,32,2.95
485,Kontagora,NG,10.40,5.47,86.97,42,15,3.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key

gmaps.configure(api_key = g_key)

In [6]:
# Convert 'Lat' and 'Lng' and Humidity to float and Store 

locations = cities_data[["Latitude", "Longitude"]].astype(float)

Humidities = cities_data["Humidity"].astype(float)

In [7]:
# Create a poverty Heatmap layer

fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = Humidities, 
                                 dissipating = False, 
                                 opacity = 0.8,
                                 max_intensity=200,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
narrowed_city_df = cities_data.loc[
                                    (cities_data["Max Temp"] < 85) & 
                                    (cities_data["Max Temp"] > 65) & 
                                    (cities_data["Wind Speed"] < 10) & 
                                    (cities_data["Cloudiness"] < 10)
                                  ].dropna()
narrowed_city_df = narrowed_city_df.sort_index()
narrowed_city_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
City_ID,,,,,,,,
5,Rajpipla,IN,21.87,73.50,81.86,66,0,4.09
79,Fatehpur,IN,25.93,80.80,78.31,49,0,6.51
106,Namibe,AO,-15.20,12.15,77.74,69,0,2.01
113,Dingle,PH,11.00,122.67,80.06,82,4,3.11
129,Faya,SA,18.39,42.45,66.20,88,8,1.41
160,Sarangani,PH,5.40,125.46,83.70,74,3,7.72
168,Kidal,ML,18.44,1.41,84.79,25,3,1.61
183,Anisoc,GQ,1.87,10.77,71.76,95,7,1.34
184,Mangrol,IN,21.12,70.12,79.95,88,0,9.57


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates

hotel_df = narrowed_city_df[["City", "Country", "Max Temp", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Latitude,Longitude,Hotel Name
City_ID,,,,,,
5,Rajpipla,IN,81.86,21.87,73.50,
79,Fatehpur,IN,78.31,25.93,80.80,
106,Namibe,AO,77.74,-15.20,12.15,
113,Dingle,PH,80.06,11.00,122.67,
129,Faya,SA,66.20,18.39,42.45,
160,Sarangani,PH,83.70,5.40,125.46,
168,Kidal,ML,84.79,18.44,1.41,
183,Anisoc,GQ,71.76,1.87,10.77,
184,Mangrol,IN,79.95,21.12,70.12,


In [10]:
# Build the base URL for google apis
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for a hotel

params = { "radius": 5000,
           "types": "lodging",
           "key": g_key
         }

# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    
    # get latitude , longitude from df
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    response = requests.get(base_url, params=params)
    
    # Convert to json
    response_json = response.json()
    
    # Grab the first hotel from the results and store the name
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Max Temp,Latitude,Longitude,Hotel Name
City_ID,,,,,,
5,Rajpipla,IN,81.86,21.87,73.50,Government Guest House
79,Fatehpur,IN,78.31,25.93,80.80,City Club
106,Namibe,AO,77.74,-15.20,12.15,Chik-Chik Namibe
113,Dingle,PH,80.06,11.00,122.67,Camp Pasica
129,Faya,SA,66.20,18.39,42.45,مقبرة ال مجمل
160,Sarangani,PH,83.70,5.40,125.46,Ballistic Islet
168,Kidal,ML,84.79,18.44,1.41,
183,Anisoc,GQ,71.76,1.87,10.77,
184,Mangrol,IN,79.95,21.12,70.12,Hotel The Royal Honours


In [11]:
# Create list of hotels to use for hover_text

hotels = []

for hotel in hotel_df["Hotel Name"]:
    hotels.append(hotel)

print(hotels)

['Government Guest House', 'City Club', 'Chik-Chik Namibe', 'Camp Pasica', 'مقبرة ال مجمل', 'Ballistic Islet', '', '', 'Hotel The Royal Honours', 'Taloa Heights', 'Gg. Munasir Homestay', 'Sheraton Gambia Hotel Resort & Spa', 'Taitung Bali Suites Hotel', 'Pousada Bela Vista', 'Hotel Kumar International', 'Circuit House, Siaha', 'Guest Hotel', '', 'bador sepret', 'Hotel Somisetty Landmark Konaseema', 'لی پوش jean center', 'Akila Hotel', 'CH Hotel - Mpanda', 'สวนหนิง-อัศ', 'Coral Sands Hotel']


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info, hover_text = hotels)
fig.add_layer(marker_layer)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))